<a href="https://colab.research.google.com/github/akitim/SkillFactory_projects/blob/main/Project_2_EDA_Timofey_Akimkin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Загрузка библиотек

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind
 
pd.set_option('display.max_rows', 50) # показывать больше строк
pd.set_option('display.max_columns', 50) # показывать больше колонок
 
import sys
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
sys.path.clear
sys.path.append('/content/gdrive/MyDrive/SkillFactory')

Mounted at /content/gdrive


#Считывание датафрейма из файла

In [3]:
cacao = pd.read_csv('/content/gdrive/MyDrive/SkillFactory/stud_math.csv')
display(cacao.sample(10))

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,"studytime, granular",higher,internet,romantic,famrel,freetime,goout,health,absences,score
50,GP,F,16,U,LE3,T,2.0,2.0,services,services,course,mother,3.0,2.0,0.0,no,yes,yes,no,yes,-6.0,yes,yes,no,4.0,3.0,3.0,4.0,2.0,65.0
94,GP,M,15,U,LE3,T,2.0,2.0,services,health,reputation,mother,1.0,4.0,0.0,no,yes,no,yes,yes,-12.0,yes,yes,no,NaN,3.0,4.0,4.0,6.0,70.0
283,GP,F,18,U,GT3,T,1.0,1.0,other,other,home,mother,2.0,2.0,0.0,yes,NaN,no,yes,yes,-6.0,yes,yes,no,5.0,NaN,4.0,4.0,4.0,NaN
124,GP,F,16,U,GT3,T,2.0,2.0,other,other,home,mother,1.0,2.0,0.0,no,no,yes,no,yes,-6.0,yes,yes,yes,5.0,4.0,4.0,5.0,0.0,40.0
368,MS,F,18,U,GT3,T,2.0,3.0,at_home,services,course,father,2.0,1.0,0.0,no,yes,yes,no,yes,-3.0,yes,yes,yes,5.0,2.0,3.0,4.0,0.0,50.0
255,GP,M,17,U,LE3,T,1.0,1.0,health,other,course,mother,2.0,1.0,1.0,no,yes,no,yes,yes,-3.0,yes,yes,no,4.0,4.0,4.0,5.0,2.0,40.0
379,MS,F,17,R,GT3,T,3.0,1.0,at_home,other,reputation,mother,1.0,2.0,0.0,no,yes,yes,yes,no,-6.0,yes,yes,no,4.0,5.0,4.0,1.0,17.0,50.0
388,MS,F,18,U,LE3,T,3.0,1.0,teacher,services,course,mother,1.0,2.0,0.0,no,yes,yes,no,yes,-6.0,yes,yes,no,4.0,3.0,4.0,1.0,0.0,40.0
191,GP,F,17,U,GT3,T,1.0,1.0,at_home,services,course,mother,1.0,2.0,0.0,no,no,no,yes,yes,-6.0,yes,NaN,no,5.0,3.0,3.0,3.0,0.0,45.0
200,GP,F,16,U,GT3,T,4.0,3.0,health,other,home,mother,1.0,2.0,0.0,no,yes,NaN,yes,yes,-6.0,yes,yes,no,4.0,3.0,NaN,2.0,2.0,80.0


#Предобработка

In [ ]:

def hasletornum(s):
    ls = [ch for ch in s]
    return any(list(map(lambda x: x.isalnum(), ls)))
